## Traiter des corpus

In [1]:
!uv pip install langchain-text-splitters

Using Python 3.11.14 environment at: /Users/elliec/Desktop/cy/scripts_perso/git_perso311
Audited 1 package in 5ms


In [2]:
import re
import json

with open("doc.txt", "r", encoding="utf-8") as f:
    text = f.read()
text = re.split(r"Titre original", text, maxsplit=1, flags=re.IGNORECASE)[0]
m = re.search(r"INDEX DES PERSONNAGES", text, flags=re.IGNORECASE)
if not m:
    raise ValueError("Can't find INDEX DES PERSONNAGES")

people_raw = text[m.start():]
parts = re.split(r"\bPROLOGUE\b", people_raw, maxsplit=1, flags=re.IGNORECASE)
people_raw = parts[0]

lines = [re.sub(r"^\s+", "", l) for l in people_raw.splitlines()]
lines = [l for l in lines if l.strip() != ""]
families = []
current_family = None

for line in lines:
    if re.match(r"^[A-Z].*:", line):
        if "INDEX DES PERSONNAGES" in line:
            continue
        current_family = {
            "family": line.replace(" :", "").strip(),
            "members": []
        }
        families.append(current_family)
    else:
        if current_family:
            current_family["members"].append(line)

with open("characters.json", "w", encoding="utf-8") as f:
    json.dump(families, f, ensure_ascii=False, indent=2)
families[:7] 

[{'family': 'LA FAMILLE CERULLO (LA FAMILLE DU CORDONNIER)',
  'members': ['Fernando Cerullo, cordonnier.',
   'Nunzia Cerullo, mère de Lila.',
   'Raffaella Cerullo, que tout le monde appelle Lina, sauf Elena qui l’appelle Lila.',
   'Rino Cerullo, grand frère de Lila, cordonnier lui aussi.',
   'Rino sera aussi le prénom d’un des enfants de Lila.',
   'Autres enfants.']},
 {'family': 'LA FAMILLE GRECO (LA FAMILLE DU PORTIER DE MAIRIE)',
  'members': ['Elena Greco, dite Lenuccia ou Lenù. C’est l’aînée, et après elle viennent Peppe, Gianni et Elisa.',
   'Le père est portier à la mairie.',
   'La mère reste au foyer.']},
 {'family': 'LA FAMILLE CARRACCI (LA FAMILLE DE DON ACHILLE)',
  'members': ['Don Achille Carracci, l’ogre des contes.',
   'Maria Carracci, femme de Don Achille.',
   'Stefano Carracci, fils de Don Achille, épicier dans l’épicerie familiale.',
   'Pinuccia et Alfonso Carracci, les deux autres enfants de Don Achille.']},
 {'family': 'LA FAMILLE PELUSO (LA FAMILLE DU ME

In [3]:
with open("doc.txt", "r", encoding="utf-8") as f:
    text = f.read()
text = re.split(r"Titre original", text, maxsplit=1, flags=re.IGNORECASE)[0]
m = re.search(r"INDEX DES PERSONNAGES", text, flags=re.IGNORECASE)
text_after_people = text[m.end():]
parts = re.split(r"\bPROLOGUE\b", text_after_people, maxsplit=1, flags=re.IGNORECASE)
if len(parts) < 2:
    raise ValueError("can't fin PROLOGUE")
main_text = "PROLOGUE\n" + parts[1]
split_result = re.split(r"\n\s*(PROLOGUE|\d+)\s*\n", main_text)
chapters_json = []
i = 1
chapter_num = 0
while i < len(split_result):
    title = split_result[i]
    body = split_result[i+1]
    paragraphs = [p.strip() for p in re.split(r"\n\s*\n", body) if p.strip() != ""]

    chapters_json.append({
        "number": chapter_num,
        "title": title,
        "paragraphs": paragraphs
    })
    chapter_num += 1
    i += 2

with open("chapters.json", "w", encoding="utf-8") as f:
    json.dump(chapters_json, f, ensure_ascii=False, indent=2)
chapters_json[:4] 


[{'number': 0,
  'title': '1',
  'paragraphs': ['Ce matin Rino m’a téléphoné, j’ai cru qu’il voulait encore de l’argent et me suis préparée à le lui refuser. Mais le motif de son appel était tout autre : sa mère avait disparu.',
   '« Depuis combien de temps ?',
   '— Quinze jours.',
   '— Et c’est maintenant que tu m’appelles ? »',
   'Mon ton a dû lui paraître hostile ; pourtant je n’étais ni en colère ni indignée, juste un tantinet sarcastique. Il a tenté de répliquer mais n’a pu émettre qu’une réponse confuse, gênée, moitié en dialecte et moitié en italien. Il s’était mis dans la tête, m’a-t-il expliqué, que sa mère était en vadrouille quelque part dans Naples, comme d’habitude.',
   '« Même la nuit ?',
   '— Tu sais comment elle est.',
   '— D’accord, mais quinze jours d’absence, tu trouves ça normal ?',
   '— Ben oui. Ça fait longtemps que tu ne l’as pas vue, c’est encore pire : elle n’a jamais sommeil, elle va et vient, elle fait tout ce qui lui passe par la tête. »',
   'Il ava

In [4]:
chunks = []
with open("characters.json", "r", encoding="utf-8") as f:
    characters = json.load(f)
for fam in characters:
    chunk_text = fam["family"] + "\n" + "\n".join(fam["members"])
    chunks.append({
        "type": "character",
        "family": fam["family"],
        "text": chunk_text
    })
with open("chapters.json", "r", encoding="utf-8") as f:
    chapters = json.load(f)
for chap in chapters:
    for para in chap["paragraphs"]:
        chunks.append({
            "type": "chapter",
            "chapter": chap["number"],
            "title": chap["title"],
            "text": para
        })
with open("chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)
len(chunks), chunks[:4]

(1927,
 [{'type': 'character',
   'family': 'LA FAMILLE CERULLO (LA FAMILLE DU CORDONNIER)',
   'text': 'LA FAMILLE CERULLO (LA FAMILLE DU CORDONNIER)\nFernando Cerullo, cordonnier.\nNunzia Cerullo, mère de Lila.\nRaffaella Cerullo, que tout le monde appelle Lina, sauf Elena qui l’appelle Lila.\nRino Cerullo, grand frère de Lila, cordonnier lui aussi.\nRino sera aussi le prénom d’un des enfants de Lila.\nAutres enfants.'},
  {'type': 'character',
   'family': 'LA FAMILLE GRECO (LA FAMILLE DU PORTIER DE MAIRIE)',
   'text': 'LA FAMILLE GRECO (LA FAMILLE DU PORTIER DE MAIRIE)\nElena Greco, dite Lenuccia ou Lenù. C’est l’aînée, et après elle viennent Peppe, Gianni et Elisa.\nLe père est portier à la mairie.\nLa mère reste au foyer.'},
  {'type': 'character',
   'family': 'LA FAMILLE CARRACCI (LA FAMILLE DE DON ACHILLE)',
   'text': 'LA FAMILLE CARRACCI (LA FAMILLE DE DON ACHILLE)\nDon Achille Carracci, l’ogre des contes.\nMaria Carracci, femme de Don Achille.\nStefano Carracci, fils de Do

In [5]:
!uv pip install sentence-transformers

Using Python 3.11.14 environment at: /Users/elliec/Desktop/cy/scripts_perso/git_perso311
Audited 1 package in 12ms


In [6]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
def embed_chunk(text: str):
    emb = embedding_model.encode(text, normalize_embeddings=True)
    return emb.tolist()

/Users/elliec/Desktop/cy/scripts_perso/git_perso311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
with open("chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

In [9]:
embedded_chunks = []
for item in chunks:
    embedded_chunks.append({
        "text": item["text"],
        "embedding": embed_chunk(item["text"]),
        **{k: v for k, v in item.items() if k != "text"} 
    })

In [10]:
print(f"Total chunks: {len(chunks)}")
print(chunks[0])

Total chunks: 1927
{'type': 'character', 'family': 'LA FAMILLE CERULLO (LA FAMILLE DU CORDONNIER)', 'text': 'LA FAMILLE CERULLO (LA FAMILLE DU CORDONNIER)\nFernando Cerullo, cordonnier.\nNunzia Cerullo, mère de Lila.\nRaffaella Cerullo, que tout le monde appelle Lina, sauf Elena qui l’appelle Lila.\nRino Cerullo, grand frère de Lila, cordonnier lui aussi.\nRino sera aussi le prénom d’un des enfants de Lila.\nAutres enfants.'}


In [11]:
!uv pip install faiss

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.11.14 environment at: /Users/elliec/Desktop/cy/scripts_perso/git_perso311
  × No solution found when resolving dependencies:                                  
  ╰─▶ Because only faiss==1.5.3 is available and faiss==1.5.3 has no wheels
      with a matching Python ABI tag (e.g., `cp311`), we can conclude that all
      versions of faiss cannot be used.
      And because you require faiss, we can conclude that your requirements
      are unsatisfiable.

      hint: You require CPython 3.11 (`cp311`), but we only found wheels for
      `faiss` (v1.5.3) with the following Python ABI tags: `cp27m`, `cp35m`,
      `cp36m`, `cp37m`


In [ ]:
!uv pip install chroma

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.11.14 environment at: /Users/elliec/Desktop/cy/scripts_perso/git_perso311
Audited 1 package in 2ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.11.14 environment at: /Users/elliec/Desktop/cy/scripts_perso/git_perso311
Resolved 84 packages in 842ms                                        
   Building pypika==0.48.9                                             
   Building pypika==0.48.9                                     
⠙ Preparing packages... (0/50)
   Building pypika==0.48.9-------------     0 B/102.89 KiB          
⠙ Preparing packages... (0/50)
   Building pypika==0.48.9-------------     0 B/102.89 KiB          
⠙ Preparing packages... (0/50)
opentelemetry-api    ------------------------------     0 B/64.80 KiB
   Building pypika==0.48.9-------------     0 B/102.89 KiB          
⠙ Preparing packages... (0/50)
opentelemetry-api    ------------------------------     0 B/64.80 KiB
   Building pypika==0.48.9-------------     0 B/102.89 KiB          
⠙ Preparing packages... (0/50)
opentelemetry-api    ------------------------------     0 B/64.80 KiB
   Building pypika==0.48.9-------------     0 B/102.89 KiB      

In [12]:
import chromadb

client = chromadb.EphemeralClient()
collection = client.get_or_create_collection(
    name="novel",
    metadata={"hnsw:space": "cosine"} 
)

In [13]:
for i, chunk in enumerate(chunks):
    vec = embed_chunk(chunk["text"])
    metadata = {k: v for k, v in chunk.items() if k != "text"}
    collection.add(
        ids=[str(i)],
        documents=[chunk["text"]],
        embeddings=[vec],
        metadatas=[metadata]
    )

print("Done for vector library")
print("num of vectors：", collection.count())

Done for vector library
num of vectors： 1927


In [14]:
def search(query, k=5):
    q_vec = embed_chunk(query)

    results = collection.query(
        query_embeddings=[q_vec],
        n_results=k
    )
    return results

In [15]:
res = search("Qui est Lila ?", k=5)
res["documents"][0]    
res["metadatas"][0]     
res["ids"][0]           


['1260', '1436', '1783', '834', '600']

In [16]:
def retrieve(query: str, top_k: int):
    query_embedding = embed_chunk(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results["documents"][0]


In [17]:
query = "Qui est Lila ?"
retrieved_chunks = retrieve(query, 5)
for i, chunk in enumerate(retrieved_chunks):
    print(f"[{i}] {chunk}\n")

[0] — Lila, j’ai peur.

[1] Lila répondit, sérieuse :

[2] Lila s’habilla avec le plus grand soin. Nous nous rendîmes à pied jusqu’à l’immeuble où la maîtresse habitait, à deux pas de l’église. Pendant que nous montions, je me rendis compte qu’elle était très anxieuse. Moi j’étais habituée à ce trajet et à ces escaliers mais pas elle, et elle ne pipa mot. Je tournai la clef de la sonnette et entendis le pas traînant de Mme Oliviero.

[3] Au début Lila en resta étourdie, comme si elle avait oublié où elle se trouvait – les rues, le quartier, la cordonnerie. Puis, un jour en fin d’après-midi, elle apparut chez moi avec l’air de celle qui a tout compris et m’annonça :

[4] Mais Lila eut un de ses regards intenses et je vis qu’elle était absorbée par le livre que je tenais dans les mains :



In [18]:
!uv pip install google-generativeai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using Python 3.11.14 environment at: /Users/elliec/Desktop/cy/scripts_perso/git_perso311
Audited 1 package in 44ms


In [19]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash")

def generate(query: str, chunks: list[str]) -> str:
    context = "\n\n".join(chunks)
    prompt = f"""
        Tu es un assistant de lecture pour un roman.
        Réponds uniquement en te basant sur les extraits suivants.
        Question : {query}
        Extraits pertinents :
        {context}
        Règles :
        - Ne pas inventer d'informations.
        - Répondre en français.
        - S'appuyer uniquement sur les extraits fournis.
    """
    response = model.generate_content(prompt)
    return response.text

In [20]:
def rag(query: str, top_k: int = 5):
    chunks = retrieve(query, top_k)
    answer = generate(query, chunks)
    return answer

In [21]:
print(rag("Qui est Lila ?"))

D'après les extraits fournis :

*   Lila est un personnage qui peut ressentir de la peur et de l'anxiété.
*   Elle est décrite comme sérieuse.
*   Elle s'habille avec le plus grand soin.
*   Elle accompagne la narratrice à l'immeuble où habite la maîtresse.
*   Elle n'est pas habituée au trajet et aux escaliers de ce lieu, contrairement à la narratrice.
*   Elle peut être étourdie, comme si elle avait oublié où elle se trouvait.
*   Elle apparaît chez la narratrice avec un air de compréhension.
*   Elle a des regards intenses et peut être absorbée par un livre.
